<a href="https://colab.research.google.com/github/dhruba2003/supportiv_assignment/blob/main/MLE_intern_supportiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.17
!pip install torch
!pip install pandas
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-non

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

In [4]:
df= pd.read_csv("intern_screening_dataset.csv")


In [5]:
df.head()

,question,answer
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What is (are) Glaucoma ?,The optic nerve is a bundle of more than 1 mil...
2,What is (are) Glaucoma ?,Open-angle glaucoma is the most common form of...
3,Who is at risk for Glaucoma? ?,Anyone can develop glaucoma. Some people are a...
4,How to prevent Glaucoma ?,"At this time, we do not know how to prevent gl..."


In [6]:
df.shape

(16406, 2)

In [7]:
df=df.drop_duplicates(subset=["question"])

In [8]:
df.shape

(14981, 2)

In [9]:
train_df, val_df = train_test_split(df, test_size=0.3)

In [10]:
train_df['question']

4484     What are the symptoms of Congenital adrenal hy...
15150        What are the treatments for Tangier disease ?
16386               What causes Cyclic Vomiting Syndrome ?
3283     What are the treatments for Chilaiditi syndrome ?
12329           Is Beckwith-Wiedemann syndrome inherited ?
                               ...                        
5087     What are the symptoms of Orofaciodigital syndr...
13438    What are the genetic changes related to trisom...
9145     what research (or clinical trials) is being do...
2182                  Do you have information about Ostomy
4833       What are the treatments for Maffucci syndrome ?
Name: question, Length: 10486, dtype: object

In [11]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [12]:
def tokenize_function(examples_ques,examples_ans):
    inputs = tokenizer(examples_ques, padding='max_length', truncation=True, return_tensors='pt')
    labels = tokenizer(examples_ans, padding='max_length', truncation=True, return_tensors='pt')
    inputs.pop('token_type_ids', None)
    labels.pop('token_type_ids', None)
    inputs['labels'] = labels['input_ids']
    return inputs
# def tokenize_function(examples):
#     return tokenizer(examples['question'], examples['answer'], padding='max_length', truncation=True)


In [13]:
train_examples= train_df.to_numpy().tolist()
val_examples= val_df.to_numpy().tolist()

In [14]:
train_ques=[]
train_ans=[]
val_ques=[]
val_ans=[]
for it in train_examples:
  train_ques.append(str(it[0]))
  train_ans.append(str(it[1]))
for it in val_examples:
  val_ques.append(str(it[0]))
  val_ans.append(str(it[1]))

In [ ]:
train_ques

In [ ]:
train_ans

In [15]:
train_encodings = tokenize_function(train_ques,train_ans)

In [ ]:
print(train_encodings['labels'])

In [16]:
val_encodings = tokenize_function(val_ques,val_ans)

In [ ]:
#print(val_encodings)

In [17]:
import torch

class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = QADataset(train_encodings)
val_dataset = QADataset(val_encodings)

In [18]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [21]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )

In [ ]:
# print(train_dataset.encodings)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10486
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2622
<ipython-input-17-a5fa4383a576>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,6.070200
20,6.048300
30,5.470500
40,5.696600
50,5.051500
60,5.220300
70,4.825100
80,4.694100
90,4.557900
100,4.206700


In [ ]:
trainer.train_dataset[0]

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

In [ ]:
from transformers import pipeline

model = AutoModelForSequenceClassification.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')

nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

def get_answer(question):
    return nlp(question)[0]['label']

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = get_answer(user_input)
    print("Bot:", response)